# Bharath Gunasekaran 
# CMPE-258 Part 3b

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import math 
from tensorflow.keras.layers import Layer
from sklearn.manifold import TSNE
import tensorflow.keras as keras


# Creating Linear Layer

In [8]:
class Linear(keras.layers.Layer):
  """y = w.x + b"""

  def __init__(self, units=3, input_dim=3):
      super(Linear, self).__init__()
      w_init = tf.random_normal_initializer()
      self.w = tf.Variable(
          initial_value=w_init(shape=(input_dim, units), dtype='float32'),
          trainable=True)
      b_init = tf.zeros_initializer()
      self.b = tf.Variable(
          initial_value=b_init(shape=(units,), dtype='float32'),
          trainable=True)

  def call(self, inputs):
      # print(inputs.shape,self.w.shape)
      return tf.matmul(inputs, self.w) + self.b

In [9]:
class MLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(3)
        self.linear_2 = Linear(5)
        self.linear_3 = Linear(2,5)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

# Genearating Data

In [10]:
n = 200
d = 3
x = tf.random.normal(shape=(n,d))

weights_true = tf.constant([[5, 2,1],[1,2,1]], dtype=tf.float32)
weights_true = tf.transpose(weights_true)
bias_true = tf.constant([1], dtype=tf.float32)
y_true = (x ** 2) @ weights_true + x @ weights_true + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

x: (200, 3), weights: (3, 2), bias: (1,), y: (200, 2)


In [11]:
loss_fn = tf.keras.losses.MeanAbsoluteError()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
mlp = MLP()

for epoch in range(20):
# Open a GradientTape.
  with tf.GradientTape() as tape:

    # Forward pass.
    model = mlp(x)

    # Loss value
    loss = loss_fn(y_true, model)
      
  # Get gradients of weights wrt the loss.
  gradients = tape.gradient(loss, mlp.trainable_variables)

  # Update the weights of our linear layer.
  optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))


In [12]:
def plot3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [13]:
y_pred = mlp(x)
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(y_pred)
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot3d(X_red,y_true_red,y_pred_red)

X_red: (200, 2), y_true_red: (200, 1), y_pred_red: (200, 1)
